In [1]:
%load_ext autoreload
%autoreload 2
!sudo rm /tmp/libtpu_lockfile
import penzai
from penzai import pz
import jax.numpy as jnp
import jax_smi
import jax
jax_smi.initialise_tracking()
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
max_seq_len = 128
batch_size = 16

In [3]:
from micrlhf.llama import LlamaTransformer
from micrlhf.caching_llama import FoldedLlamaKVCachingTransformer
from micrlhf.flash import flashify

llama = LlamaTransformer.from_pretrained(("models/llama-3-70b-1.gguf", "models/llama-3-70b-2.gguf"),
                                         device_map="auto:mp=4", load_on_cpu=True,
                                        #  transpose_rotary=False
                                        #  transpose_rotary=True
                                         )
print("Folding and caching...")
llama, cache = FoldedLlamaKVCachingTransformer.from_uncached(llama, max_seq_len, {"batch": batch_size})
# llama = flashify(llama)

Folding and caching...


In [4]:
llama = llama.to_tpu()

In [5]:
import gc
gc.collect();

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-70B")
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
], tokenize=False)

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from micrlhf.sampling import sample
sample((llama, cache), tokenizer, prompt, batch_size=batch_size, max_seq_len=max_seq_len,
       do_sample=True, return_only_completion=True)

  0%|          | 0/69 [00:00<?, ?it/s]

[' Nice to....<|begin_of_text|> new....... A........<|begin_of_text|> systemsomeethingous Vub<|begin_of_text|> useendregion of......., ( and work............ A R. v... E. B<|begin_of_text|>',
 ' Nice to....000 and I from... A W a ( ___X, and all/ C on F F the with.................. v000.. F) B<|begin_of_text|>  U I<|begin_of_text|> second. B in and this<|begin_of_text|> get.',
 " Hello, use and A A A by this days a  - A A  ___x..............<|begin_of_text|> show value,'t get..... a p v. v v we ( to.Forms,  ( v.. m ( A E... __ the",
 ' Hello!the the v, test, A............ I..<|begin_of_text|> show a v a and get that" return.............. v.... A A A.. v. A A -..... I',
 ' Hello, that ( also.................. A A........... v do a to, be A..... ( for the D a G and set. I.swinging to  v.<|begin_of_text|> searchals.Forms.',
 ' Nice to I a I next the __ an p ( or F,\n then......... Aolumni........................ A.. A.... in000 we.Forms of and makeendregion of',
 ' Hello, return in<|begin_of_text|> V B the, thise............ v a.Forms.of.s in.println<|start_header_id|> to......................... (, test A. S<|begin_of_text|> overcome over for. to',
 ' Hello G will, Lorraineau (.................<|begin_of_text|> S a and create a I it<|begin_of_text|> review of to I.Forms are000.... v.... ( ( this<|begin_of_text|>\xa0 \xa0 ‚Ä¬pro, play a a A v A a,',
 ' Hello, a and this Is A...... v to<|begin_of_text|>  _........<|begin_of_text|> “of products Ionian.......... v a..... p R. ( the for et. p<|begin_of_text|> get A B for.println<|start_header_id|> S to a (',
 ' Nice and to... A be.swinging. A............. A.............. A ( not for at the un<|begin_of_text|> work ( p v A A ( I A it.Forms.of. a, v. v A only to',
 ' Hello, v,  T....... v a v<|begin_of_text|> ", a and this, not a for a v value are.Forms a......... v. p.Forms. v.... the do a a.. p.Forms of v... v.. v.',
 " Nice and be.println<|start_header_id|>leyit I201, a-objectal........................ A a the p ( t.Forms the be,'t as  The, now\n v..... v. the so closest to in.Forms and",
 ' Nice and process and a....... p a A and work... v a A Dthe............. A...... v.... A A A the. A. v.... v000. _ Candles could',
 ' Nice! e v, to. A000 v,\'t see.........<|begin_of_text|> re value, v a I............ v..... ( v this<|begin_of_text|> home a a G is D. – "..<|begin_of_text|> The and do...',
 ' Nice and show is f, R, A and test, V is the A000 by. and play the and my ( so a............ A......... A.. v......... G a the in U. v',
 ' Hello in<|begin_of_text|> "and........................ A in000....... v to000.<|begin_of_text|> move<|begin_of_text|>  test. A... v A a only to f. v., A A and  I.Forms']